In [205]:
import json
import numpy as np

In [206]:
def crear_tf_idf_matrix(inverted_index: dict[dict[int]], 
                        tf_log_scale: bool = True,
                        normalize_matrix: bool = True) -> tuple[np.array, list[str], list[int]]:
    """
    Calcula la matriz TF-IDF a partir de un índice invertido.

    Esta función toma un índice invertido que mapea términos a documentos y sus frecuencias, 
    y calcula la matriz TF-IDF para cada término en cada documento. La opción de usar escala 
    logarítmica para la frecuencia de términos (TF) es configurable, así como la opción de 
    normalizar la matriz resultante.

    Args:
        inverted_index (dict[dict[int]]): 
            Un diccionario donde las claves son términos (str) y los valores son diccionarios.
            Estos diccionarios internos mapean el ID del documento (int) a la frecuencia de ese 
            término en el documento.

        tf_log_scale (bool, opcional): 
            Indica si se debe aplicar escala logarítmica al cálculo de la frecuencia de términos (TF).
            Por defecto es True, lo que significa que se usará la fórmula `log10(1 + frecuencia)`. Si 
            se establece en False, se usará la frecuencia sin escala logarítmica.

        normalize_matrix (bool, opcional): 
            Indica si se debe normalizar la matriz TF-IDF resultante. La normalización se realiza por 
            filas, lo que significa que cada vector de documento se ajusta para que su norma sea 1. 
            Por defecto es True.

    Returns:
        tuple[np.array, list[str], list[int]]: 
            Retorna una tupla que contiene:
            - tf_idf_matrix (np.array): Un array bidimensional donde cada fila representa un documento 
              y cada columna representa un término. Los valores en la matriz son los pesos TF-IDF 
              correspondientes.
            - terms (list[str]): Una lista ordenada de los términos presentes en el índice invertido.
            - docs (list[int]): Una lista ordenada de los IDs de documentos únicos en el corpus.
    """

    # Obtiene y ordena los términos únicos en el índice invertido.
    terms = sorted(list(inverted_index.keys()))

    # Extrae y ordena los IDs de documentos únicos en el corpus.
    docs = {doc for docs_freq in inverted_index.values() for doc in docs_freq}
    docs = sorted(list(docs))

    # Calcula el Inverse Document Frequency (IDF) para cada término.
    # IDF = log10(N / df), donde N es el número total de documentos y df es la frecuencia de documentos 
    # que contienen el término.
    idf = {term: np.log10(len(docs) / len(doc_freq))
           for term, doc_freq in inverted_index.items()}

    # Calcula el Term Frequency (TF) para cada término en cada documento.
    # Si se aplica escala logarítmica, se usa la fórmula: TF = log10(1 + frecuencia).
    # Si no, se utiliza la frecuencia directa.
    if tf_log_scale:
        tf = {doc: {term: np.log10(
            1 + inverted_index[term].get(doc, 0)) for term in inverted_index.keys()} for doc in docs}
    else:
        tf = {doc: {term: inverted_index[term].get(
            doc, 0) for term in inverted_index.keys()} for doc in docs}

    # Calcula el TF-IDF para cada término en cada documento.
    # TF-IDF = TF * IDF para cada término en cada documento.
    tf_idf = {doc: {term: tf[doc][term] * idf[term]
                    for term in terms} for doc in docs}

    # Convierte el diccionario de TF-IDF en una matriz numpy para facilitar el procesamiento posterior.
    # Cada fila de la matriz representa un documento y cada columna un término.
    tf_idf_matrix = np.array([[tf_idf[doc][term] for term in terms] for doc in docs])
    
    # Normaliza la matriz TF-IDF, de manera que la norma de cada vector de documento sea 1.
    if normalize_matrix:
        tf_idf_matrix = tf_idf_matrix / np.linalg.norm(tf_idf_matrix, axis=1, keepdims=True)
    
    # Retorna la matriz TF-IDF, la lista de términos y la lista de documentos en el orden correspondiente.
    return tf_idf_matrix, terms, docs


In [207]:
inverted_index = json.loads(open('./output/inverted_index.json').read())

In [208]:
tf_idf_matrix, terms, docs = crear_tf_idf_matrix(inverted_index, 
                                                 tf_log_scale=False,
                                                 normalize_matrix=True
                                                 )

In [209]:
tf_idf_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01320803, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01320803, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])